# 全卷积网络

*全卷积网络*（fully convolutional network，FCN）

<center><img src="../img/fcn.svg" width="12%"></center>

1. 先使用卷积神经网络抽取图像特征
2. 通过$1\times 1$卷积层将通道数变换为类别个数
3. 通过转置卷积层将特征图的高和宽变换为输入图像的尺寸

## 构造模型

* 预训练的ResNet-18模型来提取图像特征
* 不需要：ResNet-18模型的全局平均池化层和全连接层
* 输入：$320\times 480$

  输出：$10\times 15$

  $1/32$

* $1\times1$卷积层：将输出通道数转换为21类
* 将特征图的高度和宽度增加32倍

  \begin{equation*}
  \frac{320-64+16\times2+32}{32}=10
  \end{equation*}
  
  \begin{equation*}
  \frac{480-64+16\times2+32}{32}=15
  \end{equation*} 
  
* 转置卷积层：步幅为$32$，卷积核为$64\times 64$，填充为$16$


```
num_classes = 21
net.add_module('final_conv', nn.Conv2d(512, 
                                       num_classes, 
                                       kernel_size=1))
net.add_module('transpose_conv', 
               nn.ConvTranspose2d(num_classes, 
                                  num_classes,
                                  kernel_size=64, 
                                  padding=16, stride=32))
```                                 

## 训练

* 与图像分类中的并没有本质上的不同
* 使用转置卷积层的通道来预测像素的类别

## 预测

当输入图像的高或宽无法被32整除时

* 在图像中截取多块高和宽为32的整数倍的矩形区域
* 这些区域的并集需要完整覆盖输入图像
* 当一个像素被多个区域所覆盖时，它在不同区域前向传播中转置卷积层输出的平均值可以作为`softmax`运算的输入，从而预测类别